# Automate GitGuardian Worflows With APIs

The real power of working with APIs is the ability to automate any workflow you can imagine...

Note: this python code requires that [`requests`](https://pypi.org/project/requests/) be installed in the environment

## Automation 1

### Auto-assign the newest incident to a random member of the workspace.

If you have just finished a historical scan or are working with a very large team that is constantly pushing code, then there is a good chance you will have discovered a lot of hardcoded credentials. These will show up in your GitGuardian dashboard as incidents. 

While you can manually assign these incidents to your team through the dashboard, one at a time, or multiple at once, with a little logic, you can assign them to a member of the team based on any criteria you choose. 

To demonstrate the concept, this example will simply assign the most recent triggered incident to a random member of the team.

This reference automation: 
- Checks for unassigned, triggered incidents
- Assigns the most recent incident to a random team member
- Pre-builds the needed sharing link, just in case we need to involve the developer
- Prints the updated incident to the screen for review

*Note: You need to have Manager or Owner level access to be able to share incidents*

In [ ]:
import requests
import authorization
import json
import random

# Let's build our header with the bearer token to be used with each API call
auth_headers = {'Authorization' : 'Token {}'.format(authorization.token) }

# Get our current list of TRIGGERED incidents, listed with the most recent first.

request_filters = {
    "ordering":"-date", 
    "status":"TRIGGERED"
}

response = requests.get(
    "http://api.gitguardian.com/v1/incidents/secrets", 
    request_filters, 
    headers=auth_headers
    )

# Store this JSON data in a var for ease of coding
incident_data = response.json()

# If there are no more incidents, we are done. 
if len(incident_data) <= 0:
    print("There are no more incidents to assign.")
    exit(keep_kernel=True)
else:
    # Assuming there is one, let's get the first incident's id 
    first_incident_id = incident_data[0]['id']

# To build our list of workspace members, we need a list to store the results    
user_list = []
# And a list of roles we want. We can only call the API for one role at a time.
roles_to_retrieve = ["manager", "owner"]

for role in roles_to_retrieve:
    # Grab a random workspace member's id with `role` so we can assign them this incident
    response = requests.get(
        "https://api.gitguardian.com/v1/members", 
        {"per_page":"100", "role": role},
        headers=auth_headers
        )
    # Build the list of user IDs. 
    for m in response.json():
        user_list.append(m['id'])

# print(user_list)

# This if-else statement accounts for orgs of 1 member 
# print(len(user_list))
if len(user_list) == 1:
    assignee_member_id = user_list[0]
else:
    assignee_member_id = user_list[random.randint(0, (len(user_list)- 1))]


## Assign the incident to the selected member of the org
response = requests.post(
    "https://api.gitguardian.com/v1/incidents/secrets/{}/assign".format(first_incident_id), 
    headers=auth_headers, 
    data = {"member_id" : assignee_member_id}
    )

# Get the sharing URL and print it
# It might already exist, for our incident, so let's check that first before we send another API call

if incident_data[0]['share_url']:
    print("You already shared this URL")
    print("Sharing URL is: " + str(incident_data[0]['share_url']))
else:
    response = requests.post("https://api.gitguardian.com/v1/incidents/secrets/{}/share".format(first_incident_id), headers=auth_headers, data = {"auto_healing" : "true"})
    print("Sharing URL is: " + str(response.json()['share_url'])) 


# And finally, let's show ALL the now updated details about our incident
response = requests.get(
    "https://api.gitguardian.com/v1/incidents/secrets/{}".format(first_incident_id),
    headers=auth_headers
    )

print(json.dumps(response.json(), indent=4))
    
    

In [ ]:
# ### This is an unassign call, for use when testing
response = requests.post("https://api.gitguardian.com/v1/incidents/secrets/{}/unassign".format(first_incident_id), headers=auth_headers, data = {"member_id" : assignee_member_id})
print(json.dumps(response.json(), indent=2))


## Automation 2

### Auto-assign incident to commit author if they are a member of the workspace.

There might be times, especially on a smaller team, when the developer who triggered a new incident is also a member of the workspace who can best handle the resolution. 

While you can easily look this up manually through the dashboard, you can also leverage a little coding to do the lifting for you.

This automation:
- Checks for the most recent incident
- Builds a list of workspace member emails
- Checks to see if the commit author email is also in the member list
    - If it is, it auto-assigns to that person
    - Otherwise, it tells us that the author is not a member of the workspace


In [ ]:
# # Uncomment the following 4 lines if you did not run the previous automation
# import requests
# import authorization
# import json
# auth_headers = {'Authorization' : 'Token {}'.format(authorization.token) }

# Get our current list of TRIGGERED incidents, listed with the most recent first.
request_filters = {
    "ordering":"-date", 
    "status":"TRIGGERED"
}
response = requests.get(
    "http://api.gitguardian.com/v1/incidents/secrets", 
    request_filters, 
    headers=auth_headers
    )

# Let's store this JSON data in a var for ease of coding
incident_data = response.json()

# If there are none, we are done. We can check the number of items with len()
if len(incident_data) <= 0:
    print("There are no more incidents to assign.")
    exit(keep_kernel=True)
else:
    # Assuming there is one, let's get the first incident's id 
    first_incident_id = incident_data[0]['id']

# Build our list of workspace member's emails
# note: if your list of users is larger than 100, not everyone will not be on this list.
response = requests.get(
    "https://api.gitguardian.com/v1/members",
    {"per_page":"100"},
    headers=auth_headers
    )

# Build the list of user IDs. 
user_email_list = []
for m in response.json():
    user_email_list.append(m['email'])

# Get the email of the committer from the incident data
response = requests.get(
    "https://api.gitguardian.com/v1/incidents/secrets/{}".format(first_incident_id), 
    headers=auth_headers
    )
comitter_email = response.json()['occurrences'][0]['author_info']

# Let's check that the email address does exist and matches one of the emails in our workspace member list
if comitter_email and any(comitter_email in email for email in user_email_list):
    # If yes: assign the incident to the user via their email address
    response = requests.post(
        "https://api.gitguardian.com/v1/incidents/secrets/{}/assign".format(first_incident_id), 
        headers=auth_headers, 
        data = {"email" : comitter_email}
        )
    # print the updated incident information
    print("This incident has been assigned to "+str(comitter_email)+" who also made the commit")
    print(json.dumps(response.json(), indent=4))
        
else:
    print('The commit author is not a member of the workspace. No update to the incident has been made.')    
    

## Automation 3

### Ignore an incident and mark it as a 'test credential' if all occurrences of a secret are in a /test/ directory and invalid.

While it is _ALWAYS_ a good idea to manually verify that credentials in /test/ folders are really test credentials, if it is a credential you know is not valid, then you might choose to ignore them.

This automation:
- Checks for the most recent incident
- Checks to see if: 
    - The incident is marked 'invalid'
    - All occurrences were inside paths that contain '/test'
        - If those 2 things are true - marks the incident as ignored with ignore_reason set to "test_credential"
- Prints the updated incident for review


In [ ]:
# # Uncomment the following 4 lines if you did not run the previous automation
import requests
import authorization
import json
auth_headers = {'Authorization' : 'Token {}'.format(authorization.token) }

# Get the current list of TRIGGERED incidents, listed with the most recent first.
request_filters = {
    "ordering":"-date", 
    "status":"TRIGGERED"
}
response = requests.get(
    "http://api.gitguardian.com/v1/incidents/secrets", 
    request_filters, 
    headers=auth_headers
    )

# Let's store this JSON data in a var for ease of coding
incident_data = response.json()

# If there are none, we are done. We can check the number of items with len()
if len(incident_data) == 0:
    print("There are no more incidents to assign.")
    exit()
else:
    # Assuming there is one, let's get the first incident's id 
    first_incident_id = incident_data[0]['id']    
    
# Get the full incident data, including occurrence level details
response = requests.get(
    "https://api.gitguardian.com/v1/incidents/secrets/{}".format(first_incident_id), 
    headers=auth_headers
    )

# Check if the secret is `invalid`
# If it is, make a list of the filepaths for each occurrence

filepath_list = []
if response.json()['validity'] != "invalid":
    print("The secrets are not marked as invalid!")
else:
    for m in response.json()['occurrences']:
        filepath_list.append(m['filepath'])
    print(filepath_list)

    # Check that the last step built a populated list and all the paths contain `/test/`
    # and if that passes - let's mark the incident as ignored 
    if filepath_list and all('/test/' in filepath for filepath in filepath_list):
        response = requests.post(
            "https://api.gitguardian.com/v1/incidents/secrets/{}/ignore".format(first_incident_id), 
            headers=auth_headers, 
            data = {"ignore_reason" : "test_credential"}
            )
        print(json.dumps(response.json(), indent=4))
    else:
        print("Could not verify all occurrences were in /test/ folders")



## What is next??

### You are only limited by your imagination and code

I hope this notebook has been helpful in understanding what is possible with the GitGuardian API.  
While this example used Python and is in a stand-alone Jupyter Notebook format, the API can be invoked with whatever language you choose to use and be embedded in any system where you can inject code. 

If you have other ideas for automations you want to see in this repository, I encourage you to make a pull request or open an issue. 

Always feel free to reach out to us at contact@gitguardian.com
